## Generic set up

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import tensorflow as tf
import numpy as np

In [ ]:
print(tf.__version__)

In [ ]:
print(tf.config.list_physical_devices('GPU'))

## Hello world!

In [ ]:
# Create a tensor to hold text
hello = tf.constant('Hello world!')

# Let's try printing this tensor and see what we get
print(hello)

# What we see here is both the text that is contained in the constant hello and
# some of its properties, its shape and data type.

## Computations

In [ ]:
# Create two tensors that have a constant value then add them together

# Create a constant with value 5
x = tf.constant(5.0)
# Create a constant with value 15
y = tf.constant(15.0)  

# Add the two constants together and print the result
z = x + y
print(z)

# Note it's been given a default datatype (float32) that was determined by x and
# y, a default name, and has no shape as it is a rank-0 tensor (a scalar). 
# We could also use the following command if we wanted to explicitly use the 
# tf.add function, e.g. tf.add(x, y)

In [ ]:
# Run some computations using rank 2 tensors (i.e. matrices)

# Create constant tensors
x = tf.constant([[1, 2, 3],[4, 5, 6]]) 
y = tf.constant([[6, 5, 4],[3, 2, 1]])  

# Add them together and print the result
z = tf.add(x,y)
print(z)

# Note that this output has been given a shape, as it is a rank 2 tensor and 
# the dtype has changed to int32 

## Constants vs Variables

We have seen so far that constants allow us to create tensors that have pre-determined values. We now introduce Tensorflow variables that contain values that can change during the course of program execution.

In [ ]:
# Create variables
x = tf.Variable(2, tf.int16)
y = tf.Variable(3, tf.int16)
z = tf.Variable(2, tf.int16)  
 
# Perform operations on the variables and print the result
xy = tf.add(x, y)
xyz = tf.multiply(xy, z)
print(xyz)

In [ ]:
# To update the value of a variable we can use the following:
# Check value of x
print(x)

# Update its value
x.assign(3, read_value=False)

# Print the new variable
print(x)

# Note, the docs provide the following description for the read_value parameter
# "if True, will return something which evaluates to the new value of the 
# variable; if False will return the assign op." I'm not 100% certain what that
# means yet

# Note 2, if you want to add or subtract a value from a tensorflow variable then
# you can use the assign_add and assign_sub functions

## TensorFlow and Numpy

The variables defined above have a property named numpy. This is the value that is stored in the tensor that will be used in any standard numpy operation.

In many contexts, it's possible to use numpy arrays, standard python variables, and TensorFlow tensors interchangeably.

In [ ]:
# Replace the TensorFlow variable declaration in our earlier example with 
# standard python variables.

# Create variables
x = 2
y = 3
z = 2
  
# Perform operations on the variables 
xy = tf.add(x, y)
xyz = tf.multiply(xy, z)
 
# Print the output
print(xyz)

# Note, when we do this the operation tf.add has taken in normal variables,
# converted them to tensors and then performed the operations. We can also do
# the same thing with multidimensional numpy arrays instead of scalars.

In [ ]:
# Create some numpy arrays with all the same values
x = np.full((3,3),2)
y = np.full((3,3),3)
z = np.full((3,3),2)
 
# Perform operations on the variables 
xy = tf.add(x, y)
xyz = tf.multiply(xy, z)
  
# Print the output
print(xyz)

In [ ]:
# If we want to specifically access the result as an array in numpy, we can 
# execute the following,

# Store the numpy array in a new variable
xyz_np = xyz.numpy()
 
# Print the array
print(xyz_np)

# Note, if you try and pass inconsistent shapes then tensorflow will return an 
# error stating that we're trying to perform an opteration on tensors with
# incompatible shapes.

## Eager mode vs Graph mode

TensorFlow allows you to interact with the library in one of two ways.

Using eager mode, it works like a normal imperative programming language. You type commands and they are executed immediately. The results are tensors that can be inspected directly or converted to numpy arrays. This is how we have used TensorFlow in the first examples above.

The second way is to use graph mode. Here we tell TensorFlow we want to construct a graph of operations that we will reuse over and over. To do this we have to collect our operations into function and use a function decorator to tell Python this function should be treated differentl

In [ ]:
# If we want to switch to graph mode, we have to tell tensorflow that it should 
# take the function we have defined and compile it into a graph.

# To do this we use a function decorator, tf.function. This is placed directly 
# above the function declaration.

# create the function using the function decorator
@tf.function
def myfunction(X_in,Y_in):
    # some complex tensorflow code we want to repeat with different values
    X_in = tf.expand_dims(X_in,-1)
    Xs = tf.square(X_in)
    Xs = tf.reduce_sum(Xs, axis=-1, keepdims=True)
    dist = -2 * tf.matmul(X_in, X_in, transpose_b=True)
    dist += Xs + tf.linalg.matrix_transpose(Xs)
    K = tf.exp(-dist)
    K = K + (tf.eye(tf.shape(K)[0], dtype=tf.float64) * 1e-6)
    L = tf.linalg.cholesky(K)
    f = tf.matmul(L,Y_in)
    # end of the complex code

    return f
 
# start the timer
start = time.time()
 
# run the code once
Y = np.random.normal(size=(n,1))
f_function = myfunction(X,Y)
 
# report the time
print('the code took ', time.time()-start, ' seconds to run')

In [ ]:
# Loop through this and measure how long it takes to run

start = time.time()
for i in range(5000):
    Y = np.random.normal(size=(n,1))
    f_function = myfunction(X,Y)
 
# report the time
print('the loop took ', time.time()-start, ' seconds to run')

The code takes a while to run the first time but is then more efficient after that. 

This is because the first time the code runs it has to compile the graph and this takes a bit of extra time. Once its done that it can reuse the same graph and so for the next iterations its faster.